<a href="https://colab.research.google.com/github/avahuu/nyt_archived/blob/main/nyt_archived.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import re
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
news_df = pd.read_json("/content/drive/MyDrive/github/nyt_archived/data/nyt_articles.txt")
news_df

/tmp/ipython-input-1777245084.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  news_df = pd.read_json("/content/drive/MyDrive/github/nyt_archived/data/nyt_articles.txt")


ValueError: Expected object or value

In [ ]:
dates = pd.DataFrame(index=news_df.index)

dates["Year"] = news_df["pub_date"].str.extract(r"^(\d{4})", expand=False)
dates["Month"] = news_df["pub_date"].str.extract(r"-(\d{2})-", expand=False)
dates["Minute"] = news_df["pub_date"].str.extract(r"T\d{2}:(\d{2}):", expand=False)

dates = dates.astype(int)

news_df_dates = news_df.merge(
    right=dates,
    left_index=True,
    right_index=True
)
news_df_dates.head()

In [ ]:
quarters = ((news_df_dates["Month"] - 1) // 3 + 1).astype(str)

# Concatenate Year + "Q" + Quarter
news_df_dates["Quarter"] = news_df_dates["Year"].astype(str) + "Q" + quarters

In [ ]:
#investigate popularity trends of three topics: China, Zoom, Labubu, and GPT models

patterns = {
    "China":  r"\bChina\b",
    "Labubu": r"\bLabubu(s)?\b",
    "Zoom": r"\bZoom\b",
    "GPT Model": r"(?:\b[A-Za-z]+-?GPT\b|\bGPT-\d[A-Za-z]?\b)",
}

for col, pat in patterns.items():
    news_df_dates[col] = (
        news_df_dates["lead_paragraph"]
        .str.count(pat)               # regex by default, case-sensitive
        .fillna(0)
        .astype(int)
    )

news_df_dates

In [ ]:
# The values are the number of articles that mentioned each topic in each quarter

topics = ["China","Labubu", "Zoom", "GPT Model"]

def num_mentioned(ser):
    return (ser > 0).sum()

topic_mentions = (
    news_df_dates
    .groupby("Quarter")[topics]
    .agg(num_mentioned)
    .sort_index()
)

# Year 2023 records
topic_mentions

In [ ]:
#visualization

plt.figure(figsize=(12, 5))

for topic in topics:
    sns.lineplot(data=topic_mentions, x="Quarter", y=topic, marker="o", label=topic)

plt.xticks(rotation=60)
plt.yticks()
plt.ylabel("Number of Articles")
plt.xlabel("Quarter")
plt.title("Number of Articles Released (2019-2024)")
plt.gcf().set_facecolor('white')
plt.show()

In [ ]:
# Sentiment Analysis
from transformers import pipeline
model_checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"

sentiment_analysis = pipeline("sentiment-analysis", model=model_checkpoint)

import torch
from transformers import pipeline

# 2)
model_checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

device = 0 if torch.cuda.is_available() else -1

sentiment_analysis = pipeline(
    task="sentiment-analysis",
    model=model_checkpoint,
    device=device
)

texts = news_df_dates["lead_paragraph"].fillna("").tolist()

results = sentiment_analysis(
    texts,
    truncation=True,
    max_length=256,
    batch_size=32
)

nyt_sentiments = pd.DataFrame(results, index=news_df_dates.index)

nyt_sentiments["sentiment_sign"] = nyt_sentiments["label"].map({"POSITIVE": 1, "NEGATIVE": -1})
nyt_sentiments["score_signed"] = nyt_sentiments["score"] * nyt_sentiments["sentiment_sign"]


In [ ]:
sentiment = nyt_sentiments

In [ ]:
sentiment["article_sentiment"] = (
    (2*(sentiment["label"] == "POSITIVE") - 1) * sentiment["score"]
)
news_df_sentiment = news_df_dates.merge(sentiment, on="web_url")
news_df_sentiment = news_df_sentiment.drop(columns=["label", "score"])